In [9]:
import numpy as np

class Node:
  # Defining the class for tree nodes
  def __init__(self, feature_index = None, threshold = None, left = None, right = None, *, value = None ):
    self.feature_index = feature_index  # index of the feature used for split
    self.threshold = threshold          # threshold for the split
    self.left = left                    # left child
    self.right = right                  # right child
    self.value = value                  # value (final prediction) if it's a leaf node

class DecisionTreeClassifier:
  # Defining the class for the classifier
  def __init__(self, max_depth = 5, min_samples_split = 2):
    self.root = None
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split

  # method to calculate Gini impurity
  # (this is what is used for optimization by the greedy algorithm of DTs)
  def gini(self, y):
    classes = np.unique(y)
    impurity = 1.0
    for clas in classes:
      p = np.sum(y==clas) / len(y)
      impurity -= p**2
    return impurity

  # Method to find the best split (greedy algorithm)
  def best_split(self, X, y):
    best_gain = -1
    split_idx, split_thresh = None, None
    current_impurity = self.gini(y) # initial impurity is impurity of the whole dataset
    n_features = X.shape[1]

    for f_indx in range(n_features):
      # Loop through all the features
      thresholds = np.unique(X[:, f_indx])
      for thresh in thresholds:
        # Loop through possible splits in the features
        left_idx = np.where(X[:, f_indx] <= thresh)[0]
        right_idx = np.where(X[:, f_indx] > thresh)[0]

        if len(left_idx) == 0 or len(right_idx) == 0:
            continue

        left_impurity = self.gini(y[left_idx])
        right_impurity = self.gini(y[right_idx])
        n = len(y)
        n_left, n_right = len(left_idx), len(right_idx)
        # Calculate impurity based on this split
        weighted_impurity = (n_left / n) * left_impurity + (n_right / n) * right_impurity
        # Calculate Info gain, (we want to minimize whole impurity of nodes by maximizing Info Gain)
        info_gain = current_impurity - weighted_impurity

        if info_gain > best_gain:
          # If this split has the best info gain, then this is the best split
          best_gain = info_gain
          split_idx = f_indx
          split_thresh = thresh

    return split_idx, split_thresh, best_gain

  def build_tree(self, X, y, depth = 0):
    # Define recursive function to build trees
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))

    # condition for stopping
    if depth >= self.max_depth or n_labels == 1 or n_samples <self.min_samples_split:
      leaf_value = self.majority_class(y)
      return Node(value=leaf_value)

    # else we find the best split
    feature_index, threshold, gain = self.best_split(X,y)
    if gain == 0:
      # If the gain is 0, then make a leaf node
      leaf_value = self.majority_class(y)
      return Node(value = leaf_value)

    # else we update indexes of remaining data for more splitting
    left_idx = np.where(X[:, feature_index] <= threshold)[0]
    right_idx = np.where(X[:, feature_index] > threshold)[0]

    # we call build_function again to continue until the stopping conditions are met
    left_subtree = self.build_tree(X[left_idx], y[left_idx], depth + 1)
    right_subtree = self.build_tree(X[right_idx], y[right_idx], depth + 1)
    return Node(feature_index, threshold, left_subtree, right_subtree)

## function for finding majority class for leaf nodes0
  def majority_class(self, y):
    counts = np.bincount(y)
    return np.argmax(counts)

  def fit(self, X, y):
    # Fit tree, with first build as the root
    self.root = self.build_tree(X,y)

  def _predict(self, node, x):
    if node.value is not None:
      return node.value
    if x[node.feature_index] <= node.threshold:
      return self._predict(node.left, x)
    else:
      return self._predict(node.right, x)

  def predict(self, data):
    return np.array([self._predict(self.root, x) for x in data])



In [22]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate a simple binary classification dataset
X, y = make_classification(
    n_samples=200, n_features=2, n_informative=2, n_redundant=0,
    n_clusters_per_class=1, random_state=42
)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = DecisionTreeClassifier(max_depth = 7, min_samples_split = 10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Decision Tree Accuracy: {acc:.2f}")

Decision Tree Accuracy: 0.88


In [12]:
# Noice